In [168]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [169]:
#reading the original dataset
all_tables = pd.read_excel("DB_SARA_01.xlsx", sheet_name=None)

In [170]:
df_graduates = all_tables["TITULADOS"]
df_programs = all_tables["PROGRAMAS"]
df_students_courses = all_tables["QRY_EST_x_MAT"]
df_courses = all_tables["QRY_MAT_EXP"]  #
df_courses2 = all_tables["QRY_MATREC_EXP"]    #
df_students_info = all_tables["QRY_ESTUDIANTS"]
df_courses2.columns=df_courses2.iloc[0]
df_courses2 = df_courses2.iloc[1:]

In [171]:
#explor in the number of all the students and graduated ones 
graduated_student_code = list(df_graduates["CODI_EST"]) 
all_students_code = list(df_students_info["Num_id"])
print("Graduated students: ", len(set(graduated_student_code)))  #When you apply the set() function to a list, it removes all duplicate elements and returns a new set with unique elements.
print("All students: ",len(set(all_students_code)))

Graduated students:  723
All students:  2130


In [172]:
len(df_students_courses[df_students_courses["CODI_EST"].apply(lambda x: x in graduated_student_code)]["CODI_EST"].unique())

723

In [173]:
len(df_students_courses[df_students_courses["CODI_EST"].apply(lambda x: x in all_students_code)]["CODI_EST"].unique())

2130

In [174]:
students_with_courses = df_courses[df_courses["CODI_EST"].apply(lambda x: x in all_students_code)]["CODI_EST"].unique()
print("Missing students in the first table: ", len(all_students_code)-len(students_with_courses))

Missing students in the first table:  106


In [175]:
not_courses = [i for i in all_students_code if i not in students_with_courses]
print("Number of students missing in the first table that are in the sencond: ",len(df_courses[df_courses["CODI_EST"].apply(lambda x: x in not_courses)]["CODI_EST"].unique()))

Number of students missing in the first table that are in the sencond:  0


In [176]:
#merging two datasets and define a new column 'IS_GRADUATED'.
df_students_courses["CODI_PLA"] = df_students_courses["CODI_PRG"].apply(lambda x: list(df_programs[df_programs["CODI_PRG"]==x]["CODI_PLA"])[0])  #This code adds a new column called "CODI_PLA" to df_students_courses. The values in this new column are derived from df_programs
df_students_courses["IS_GRADUATED"] = df_students_courses["CODI_EST"].apply(lambda x: x in graduated_student_code)
df_students_courses

,CODI_EST,CODI_ANY_ACAD,CODI_PRG,CODI_MAT,CODI_PLA,IS_GRADUATED
0,100,1415,1027,19343,87,True
1,100,1516,1182,20719,87,True
2,588,1213,798,18413,87,False
3,588,1314,927,18696,87,False
4,588,1415,1027,19509,87,False
...,...,...,...,...,...,...
6309,114531,2122,2626,63872,208,False
6310,114532,2122,2626,64360,208,False
6311,114533,2122,2626,63814,208,False
6312,114534,2122,2639,63764,210,False


In [177]:
df_students_courses['CODI_EST'].nunique()

2130

In [178]:
#merging two datasets and define a new column 'drop out'
def get_drop_outs(codi_est, cod_any, is_graduated, df):
    if is_graduated or cod_any == 2122:
        return False
    elif len(df.loc[df["CODI_EST"]==codi_est][df["CODI_ANY_ACAD"]==2122])> 0:
        return False 
    else: 
        return True
    
df_students_courses["IS_DROP_OUT"] = df_students_courses.apply(lambda x: 
                                     get_drop_outs(x.CODI_EST,x.CODI_ANY_ACAD,
                                     x.IS_GRADUATED, df_students_courses), axis=1)
df_students_courses

,CODI_EST,CODI_ANY_ACAD,CODI_PRG,CODI_MAT,CODI_PLA,IS_GRADUATED,IS_DROP_OUT
0,100,1415,1027,19343,87,True,False
1,100,1516,1182,20719,87,True,False
2,588,1213,798,18413,87,False,True
3,588,1314,927,18696,87,False,True
4,588,1415,1027,19509,87,False,True
...,...,...,...,...,...,...,...
6309,114531,2122,2626,63872,208,False,False
6310,114532,2122,2626,64360,208,False,False
6311,114533,2122,2626,63814,208,False,False
6312,114534,2122,2639,63764,210,False,False


The dataset above is the education information of all the students

In [179]:
#get the useful informations of df_students_info dataset
df_students_info.rename({'Num_id': 'CODI_EST'}, axis=1, inplace=True)
df_students_info= df_students_info[["CODI_EST", "Data_naix","Denom", "Sexe", "Nacionalitat", "Comunitat","Pais"]]

In [180]:
df_students_info['CODI_EST'].nunique()

2130

In [181]:
#merging two datasets and drop unnecessary columns
df_students_data = df_students_courses[["CODI_EST","CODI_ANY_ACAD","CODI_PRG","CODI_MAT","CODI_PLA","IS_GRADUATED","IS_DROP_OUT"]].drop_duplicates().merge(df_students_info, how='inner', on=["CODI_EST"])
droplist=['CODI_ANY_ACAD','CODI_MAT', 'CODI_PLA', ]
df_students_data= df_students_data.drop(droplist, axis=1)
df_students_data['CODI_EST'].nunique()

2130

In [182]:
#Seperate the columns of the courses in the first dataset of the students' grades
dummies = pd.get_dummies(df_courses["Denom"])
prueba = df_courses.join(dummies)

for col in dummies.columns:
    for index, row in prueba[prueba[col]==1].iterrows():
        prueba.loc[index, col] = row.Not_def_n
prueba = prueba[["CODI_PLA","CODI_EST","CODI_ANY_ACAD","CODI_TORN","CR"] + list(dummies.columns)]
prueba.loc[prueba["CODI_TORN"]==1, "CODI_TORN"]=0
prueba.loc[prueba["CODI_TORN"]==2, "CODI_TORN"]=1
df_courses_joined = prueba.groupby(["CODI_PLA","CODI_EST","CODI_ANY_ACAD"], as_index=False).sum() 
df_courses_joined['CODI_EST'].nunique()

2024

In [183]:
#Seperate the columns of the courses in the second dataset of the students' grades
dummies = pd.get_dummies(df_courses2["Denom"])
prueba = df_courses2.join(dummies)
for col in dummies.columns:
    for index, row in prueba[prueba[col]==1].iterrows():
        prueba.loc[index, col] = row.Not_def_n
prueba = prueba[["CODI_PLA","CODI_EST","CODI_ANY_ACAD","CODI_TORN","Credits"] + list(dummies.columns)]
prueba["CODI_TORN"]=0
prueba["Credits"] = [int(i) for i in list(prueba["Credits"])]
df_courses2_joined = prueba.groupby(["CODI_PLA","CODI_EST","CODI_ANY_ACAD"], as_index=False).sum()     
df_courses2_joined.rename({'Credits': 'CR'}, axis=1, inplace=True)
df_courses2_joined['CODI_EST'].nunique()

334

In [184]:
df_courses_students = pd.concat([df_courses_joined, df_courses2_joined], ignore_index=True, sort=False)
df_courses_students.fillna(0, inplace=True) 
df_courses_students["CODI_PLA"] = df_courses_students["CODI_PLA"].apply(lambda x: int(x))
df_courses_students["CODI_EST"] = df_courses_students["CODI_EST"].apply(lambda x: int(x))
df_courses_students["CODI_ANY_ACAD"] = df_courses_students["CODI_ANY_ACAD"].apply(lambda x: int(x))
df_courses_students =df_courses_students.groupby(["CODI_PLA","CODI_EST","CODI_ANY_ACAD"], as_index=False).sum()
df_courses_students['CODI_EST'].nunique()

2125

In [185]:
df= df_students_data[['CODI_EST', 'CODI_PRG', 'IS_GRADUATED', 'IS_DROP_OUT', 'Data_naix','Sexe']].drop_duplicates().merge(df_courses_students, how='inner', on=["CODI_EST"])
df = df.drop_duplicates(subset=['CODI_EST','CODI_ANY_ACAD'])
df

,CODI_EST,CODI_PRG,IS_GRADUATED,IS_DROP_OUT,Data_naix,Sexe,CODI_PLA,CODI_ANY_ACAD,CODI_TORN,CR,...,Trabajo de fin de máster,Treball de Fi de Grau,Visualització de dades,CRÈDITS OPTATIUS PER PRÀCTIQUES EXTERNES-18 CR,CRÈDITS OPTATIUS RECONEGUTS PER ACTIVITAT PROFESSIONAL-6 CR,Comptabilitat avançada,Economia internacional,Pràctiques Externes (CFGS) 18,Pràctiques externes curriculars (12),Pràctiques externes curriculars (6)
0,100,1027,True,False,1978-03-31,True,87,1415,4,198,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100,1027,True,False,1978-03-31,True,87,1516,4,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,588,798,False,True,1979-03-06,False,87,1213,3,138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,588,798,False,True,1979-03-06,False,87,1314,9,54,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,588,798,False,True,1979-03-06,False,87,1415,4,24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25808,114531,2626,False,False,1984-02-15,False,208,2122,13,78,...,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25809,114532,2626,False,False,1985-02-11,False,208,2122,10,60,...,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25810,114533,2626,False,False,1994-07-14,False,208,2122,15,90,...,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25811,114534,2639,False,False,1996-03-12,False,210,2122,15,90,...,84.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [186]:
df['CODI_EST'].nunique()

2043

There are some students in the dataset whom we don't have any information about them. they are the missing values that
we removes from the dataset. So, we have 2043 students who have the information of RS.

In [187]:
df.to_excel("All_students_with_grades_by_year.xlsx",  index=False)

# Making a dataframe for RSs

In [188]:
#reading the dataset
df= pd.read_excel("All_students_with_grades_by_year.xlsx")

In [189]:
#adding a new column to the dataset to show the degree of the students
#bachelor : 1 and master:2 and no-degree: 0
degree_map = {87: 1, 174: 1, 326: 1, 331: 1, 208: 2, 210: 2}
df['Degree'] = [degree_map.get(row, 0) for row in df['CODI_PLA']]

In [190]:
#df['Degree'].value_counts().plot(kind='bar',color='b')
#plt.show()

In [191]:
#adding a new column to show the summation of the grades each student achieved in each course
df['sum']=df.iloc[:,10:158].sum(axis=1)
#To get the number of the courses of each student in each course, we add a new column titled 'len'
nonzeros=np.count_nonzero(df.iloc[:,10:158], axis=1)
df['len']= nonzeros
#by dividing the 'sum' on the 'len', the GPA of each student in each course will be achieved
df['Average of grades']= np.ceil(df['sum']/df['len'])

Now I am going to make a new dataframe of the useful informations of students for building the RSs.

In [192]:
#to make a binary column which shows the sex of the students
def get_sex_by_student(df):
    sex = df.groupby('CODI_EST')['Sexe'].mean()
    return sex.to_frame().rename(columns={'Sexe': 'sex'})
fra2 = get_sex_by_student(df)

In [193]:
#fra2['sex'].value_counts()

In [194]:
#making two columns to show the average of grades of bachelor and master in different columns for each student
def get_gpa_by_program(df, program):
    completed_program = df.loc[df['CODI_PLA'] == program]
    return completed_program.groupby('CODI_EST')['Average of grades'].mean()

program_codes = [87, 174, 326, 331, 208, 210]
gpa_list = [get_gpa_by_program(df, program) for program in program_codes]
    
    
    
# Combine the list of GPAs into a single dataframe
fra3 = pd.concat(gpa_list, axis=1)

# Rename the columns to reflect the degree program
fra3.columns = ["GPA_B1", "GPA_B2", "GPA_B3", "GPA_B4", "GPA_M1", "GPA_M2"]

# Calculate the average GPA for each student across all bachelor degree programs
fra3['Average of grades- Bachelor'] = fra3[["GPA_B1", "GPA_B2", "GPA_B3", "GPA_B4"]].mean(axis=1)

# Calculate the average GPA for each student across all master degree programs
fra3['Average of grades- Master'] = fra3[["GPA_M1", "GPA_M2"]].mean(axis=1)

# Drop the individual degree program columns
fra3 = fra3.drop(["GPA_B1", "GPA_B2", "GPA_B3", "GPA_B4", "GPA_M1", "GPA_M2"], axis=1)

In [195]:
#making two columns to show the total credits of bachelor and master in different columns for each student
def get_cr_by_program(df, program):
    completed_program = df.loc[df['CODI_PLA'] == program]
    return completed_program.groupby('CODI_EST')['CR'].sum()

program_codes = [87, 174, 326, 331, 208, 210]
cr_list = [get_cr_by_program(df, program) for program in program_codes]

# Combine the list of GPAs into a single dataframe
fra4 = pd.concat(cr_list, axis=1)

# Rename the columns to reflect the degree program
fra4.columns = ["CR_B1", "CR_B2", "CR_B3", "CR_B4", "CR_M1", "CR_M2"]

# Calculate the average GPA for each student across all bachelor degree programs
fra4['total CR- Bachelor'] = fra4[["CR_B1", "CR_B2", "CR_B3", "CR_B4"]].sum(axis=1)

# Calculate the average GPA for each student across all master degree programs
fra4['total CR- Master'] = fra4[["CR_M1", "CR_M2"]].sum(axis=1)

# Drop the individual degree program columns
fra4 = fra4.drop(["CR_B1", "CR_B2", "CR_B3", "CR_B4", "CR_M1", "CR_M2"], axis=1)

In [196]:
fra4 = pd.concat([fra2, fra3,fra4], axis=1)

In [197]:
#adding a new binary column 'IS_GRADUATED' and 'IS_DROP_OUT'. 1 means is graduated and 0 means not graduated
def graduation_status(df1, column_name, new_column_name):
    grouped = df.groupby('CODI_EST')[column_name].mean()
    new_df = grouped.to_frame()
    new_df.columns = [new_column_name]
    return df1.join(new_df, on='CODI_EST')

# example usage
fra4 = graduation_status(fra4, 'IS_GRADUATED', 'IS_GRADUATED')
fra4 = graduation_status(fra4, 'IS_DROP_OUT', 'IS_DROP_OUT')

In [198]:
#adding the 'degree' column
def add_degree_column(df):
    # create a new column to indicate degree type (Master or Bachelor)
    df['degree'] = df['Average of grades- Master'].notna().replace({True: 'Master', False: "Bachelor"})
    # convert degree column from string to integer
    df['degree'] = df['degree'].replace(['Master'], 2)
    df['degree'] = df['degree'].replace(['Bachelor'], 1)
    return df
fra4 = add_degree_column(fra4)

In [199]:
#make a new column that in each row it provides the study programs of each student in bacheor. Some of the students have 2 
#study program for their bachelor
def separate_programs(df1, fra):
    # create a groupby object by 'CODI_EST' column
    group = df1.groupby('CODI_EST')
    
    # create a new column for the study programs of each student in bachelor
    df2 = group.apply(lambda x: list(set(x['CODI_PLA'].unique()) & set([87,174, 326,331])))
    fra5 = pd.DataFrame(df2)
    fra['study_program_code_Bachelor'] = fra5[0].values
    
    # create a new column for the study programs of each student in master
    df3 = group.apply(lambda x: list(set(x['CODI_PLA'].unique()) & set([208,210])))
    fra5 = pd.DataFrame(df3)
    fra['study_program_code_Master'] = fra5[0].values
    
    # separate the columns for the elements of the list for the study program in bachelor column
    fra[['study_program_code_Bachelor1', 'study_program_code_Bachelor2']] = pd.DataFrame(fra.study_program_code_Bachelor.tolist(), index=fra.index)
    
    # insert a new column which is the study program of the master but not in the list format
    fra[['study_program_code_Master']] = pd.DataFrame(fra.study_program_code_Master.tolist(), index=fra.index)
    
    # to delete the columns in list format for bachelor
    fra.drop('study_program_code_Bachelor', axis=1, inplace=True)
    
    return fra
fra4 = separate_programs(df, fra4)

In [200]:
#make a new dataframe of the courses of the students and the summation of their grades in each course
def calculate_course_gpas(df):
    e_list=[]
    listt=[87,174, 326,331, 208,210 ]

    # get a list of all column names in the dataframe
    all_columns = df.columns.tolist()

    # specify the column names to iterate over using a slice
    start_col = 10  # start at column D
    end_col = 157  # end at column F
    column_names = all_columns[start_col:end_col+1]

    # loop through the column names and assign them to variables
    for col in column_names:
        var_name = col  # assign the column name to a variable
        for i in listt:
            if i in [87,174, 326,331]:
                completed_program = df.loc[df['CODI_PLA'] == i]
                each_course_GPA= completed_program.groupby('CODI_EST')[var_name].sum()
                e_list.append(each_course_GPA)
            elif i in [208,210]:
                completed_program = df.loc[df['CODI_PLA'] == i]
                each_course_GPA=completed_program.groupby('CODI_EST')[var_name].sum()
                e_list.append(each_course_GPA)

    # create a new dataframe and transpose it to swap rows and columns
    fra6 = pd.DataFrame(e_list).T

    # remove duplicated columns
    fra6 = fra6.loc[:, ~fra6.columns.duplicated(keep='first')]
    
    return fra6

fra6 = calculate_course_gpas(df)

In [201]:
def merge_dataframes(df1, df2):
    all_columns = df2.columns.tolist()
    fra6.columns
    new_df = pd.concat([df1] + [df2[col] for col in all_columns], axis=1)
    return new_df
fra4 = merge_dataframes(fra4, fra6)

In [202]:
#replace all the nan values with 0
fra4= fra4. replace(np. nan,0) 

In [203]:
# select the rows where IS_GRADUATED is equal to 1 and study_program_code_Master is equal to 0
# Get 20 random data points where fra4['study_program_code_Master']== 100
def replace_unavailable_programs(df):
    # select the rows where IS_GRADUATED is equal to 1 and study_program_code_Master is equal to 0
    mask = (df['IS_GRADUATED'] == 1) & (df['study_program_code_Master'] == 0)
    # replace the value with "not available"
    df.loc[mask, 'study_program_code_Master'] = 100
    
    # Get 20 random data points where study_program_code_Master is equal to 100
    random_indices = df.index[df['study_program_code_Master'] == 100].tolist()
    random_indices = random.sample(random_indices, 20)
    df_random = df.loc[random_indices]

    # Replace the rows where study_program_code_Master is equal to 100 with the random samples
    df = df.loc[df['study_program_code_Master'] != 100]
    df = pd.concat([df, df_random])
    
    return df
fra4 = replace_unavailable_programs(fra4)
fra4

,sex,Average of grades- Bachelor,Average of grades- Master,total CR- Bachelor,total CR- Master,IS_GRADUATED,IS_DROP_OUT,degree,study_program_code_Master,study_program_code_Bachelor1,...,Trabajo de fin de máster,Treball de Fi de Grau,Visualització de dades,CRÈDITS OPTATIUS PER PRÀCTIQUES EXTERNES-18 CR,CRÈDITS OPTATIUS RECONEGUTS PER ACTIVITAT PROFESSIONAL-6 CR,Comptabilitat avançada,Economia internacional,Pràctiques Externes (CFGS) 18,Pràctiques externes curriculars (12),Pràctiques externes curriculars (6)
CODI_EST,,,,,,,,,,,,,,,,,,,,,
588,0.0,63.333333,0.0,216.0,0.0,0.0,1.0,1,0.0,87.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3060,1.0,54.714286,72.0,270.0,60.0,1.0,0.0,2,208.0,87.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3063,1.0,51.000000,0.0,90.0,0.0,0.0,1.0,1,0.0,87.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20033,1.0,55.000000,0.0,90.0,0.0,0.0,1.0,1,0.0,87.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45008,1.0,0.000000,0.0,42.0,0.0,0.0,1.0,1,0.0,87.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95140,1.0,51.000000,0.0,318.0,0.0,1.0,0.0,1,100.0,87.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100331,1.0,75.750000,0.0,240.0,0.0,1.0,0.0,1,100.0,326.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97847,1.0,67.800000,0.0,270.0,0.0,1.0,0.0,1,100.0,87.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [204]:
#to make the traing and test set. The traing set contains the students in the master degree whom we have the information for 
#their bachelor and their major in master. And the test set are the students who 1-are not graduated, 2-are not drop out, 3-and
# they are in bachelor who are already completing their bachelor

def create_train_test_sets(df):
    # Get students who completed Bachelor and Master program
    completed_students = df[(df['study_program_code_Master']== 210) | (df['study_program_code_Master']== 208) ] #| (df['study_program_code_Master'] == 100)
    
    # Filter the students who completed Bachelor program
    train = completed_students[completed_students['study_program_code_Bachelor1']>0.0]
    x_train = train.drop('study_program_code_Master', axis=1)
    y_train = train['study_program_code_Master']
    
    # Filter students who are not in completed Bachelor and already graduated or dropped out
    not_in_train = df[(df.isin(train) == False)]
    test= not_in_train[(not_in_train['IS_GRADUATED'] == 0) & (not_in_train['IS_DROP_OUT'] == 0) & (not_in_train['degree'] == 1)]
    
    # Remove missing values
    test = test.dropna()
    x_test = test.drop('study_program_code_Master', axis=1)
    y_test = test['study_program_code_Master']
    
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = create_train_test_sets(fra4)

In [205]:
#feature selection
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

def feature_selection(x_train, y_train, x_test):
   
    # Instantiate Lasso regularization model
    lasso_model = Lasso(alpha=0.3, max_iter=10000)

    # Train Lasso regularization model on the training set
    lasso_model.fit(x_train, y_train)

    # Use Lasso regularization model to select important features from training set
    selector = SelectFromModel(lasso_model, threshold=None, prefit=False, importance_getter="coef_")
    
    # Fit the selector on the training set with feature names
    selector.fit(x_train, y_train)
    
    # Transform the training and test sets using the selector
    x_train_selected = selector.transform(x_train)
    x_test_selected = selector.transform(x_test)
    
    # Get selected feature names
    selected_features = x_train.columns[selector.get_support()]

    # Convert the numpy arrays back to pandas dataframes
    x_train_selected_df = pd.DataFrame(x_train_selected, columns=selected_features)
    x_test_selected_df = pd.DataFrame(x_test_selected, columns=selected_features)

    return x_train_selected_df, x_test_selected_df

x_train, x_test = feature_selection(x_train, y_train, x_test)

In [206]:
from sklearn.model_selection import train_test_split

def split_data(x_train, y_train, test_size=0.2, random_state=42):
   
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=test_size, random_state=random_state)

    # Print the shapes of the resulting datasets
    print("Number of samples in the training set: {}".format(len(x_train)))
    print("Number of samples in the validation set: {}".format(len(x_val)))
    
    return x_train, x_val, y_train, y_val

x_train, x_val, y_train, y_val = split_data(x_train, y_train, test_size=0.2, random_state=42)
print("Number of samples in the test set: {}".format(len(x_test)))


Number of samples in the training set: 32
Number of samples in the validation set: 8
Number of samples in the test set: 604


In [207]:
#bootstrapping
from sklearn.utils import resample

def bootstrap_data(x_train, y_train, n_bootstrap=10, random_state=42):
    
    # List to hold new bootstrap samples
    bootstrapped_data = []

    # Generate new bootstrap samples
    for i in range(n_bootstrap):
        x_boot, y_boot = resample(x_train, y_train, replace=True, random_state=random_state+i)
        bootstrapped_data.append((x_boot, y_boot))

    # Concatenate original training data with new bootstrap samples
    x_train_boot = np.concatenate([x_train] + [data[0] for data in bootstrapped_data])
    y_train_boot = np.concatenate([y_train] + [data[1] for data in bootstrapped_data])

    return x_train_boot, y_train_boot

x_train, y_train = bootstrap_data(x_train, y_train, n_bootstrap=40, random_state=42)

# Check the shapes of the new training data
print("X_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (1312, 25)
y_train shape: (1312,)


# KNN

In [208]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

def knn_classification(x_train, y_train, x_val, y_val):
   
    # Standardize the features
    scaler = StandardScaler(with_mean=True, with_std=True)
    X_train = scaler.fit_transform(x_train)
    X_val = scaler.transform(x_val)

    # Hyperparameter tuning
    params = {'n_neighbors':[1,2,3,4,5,6,7,8,9, 10, 20, 30, 50]}
    knn = KNeighborsClassifier()
    model = GridSearchCV(knn, params, cv=5)
    model.fit(X_train, y_train)
    best_params = model.best_params_

    # Train the final model using the best hyperparameters
    model = KNeighborsClassifier(**best_params)
    model.fit(X_train, y_train)
    y_pred_knn = model.predict(X_val)
    
    # Evaluate the model on the test set
    accuracy = accuracy_score(y_val, y_pred_knn)

    return y_pred_knn, best_params, accuracy

y_pred_knn, best_params, accuracy = knn_classification(x_train, y_train, x_val, y_val)
print(y_pred_knn, best_params, accuracy)

[208. 208. 208. 210. 208. 208. 210. 210.] {'n_neighbors': 1} 0.625


# Decision Tree

In [209]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

def decision_tree_classification(x_train, y_train, x_val, y_val, feature_names=None, impurity_range=(0, 0.01), n_iter=50, cv=5, n_jobs=-1):
    
    # Define the range of min_impurity_decrease values to test
    param_dist = {'min_impurity_decrease': uniform(*impurity_range)}

    # Create a decision tree classifier
    dt = DecisionTreeClassifier()

    # Perform a randomized search with cross-validation
    random_search = RandomizedSearchCV(dt, param_distributions=param_dist, cv=cv, n_jobs=n_jobs, n_iter=n_iter)

    # Fit the randomized search on training data
    random_search.fit(x_train, y_train)

    # Create a decision tree classifier with the best hyperparameters
    clf = DecisionTreeClassifier(min_impurity_decrease=random_search.best_params_['min_impurity_decrease'])

    if feature_names is not None:
    # Train the decision tree classifier
        clf = clf.fit(x_train, y_train, feature_names=feature_names)
    else:
        clf = clf.fit(x_train, y_train)
    # Predict the response for the validation dataset
    y_pred_dt = clf.predict(x_val)

    # Compute the accuracy score of the classifier
    accuracy = metrics.accuracy_score(y_val, y_pred_dt)

    return y_pred_dt, clf, accuracy

y_pred_dt, dt_model, accuracy = decision_tree_classification(x_train, y_train, x_val, y_val, impurity_range=(0, 0.01), n_iter=50, cv=5, n_jobs=-1)
print(y_pred_dt, dt_model, accuracy)

[210. 208. 210. 208. 208. 208. 210. 210.] DecisionTreeClassifier(min_impurity_decrease=0.004040253130378467) 0.75


# Random Forest

In [210]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


def random_forest_classification(x_train, y_train, x_val, y_val):
    # Define the parameter grid
    param_grid = {
        'n_estimators': [50, 100, 150],
        'min_samples_leaf': [1, 5, 10]
    }

    # Create a random forest classifier object
    rfc_param = RandomForestClassifier(random_state=42)

    # Create a grid search object
    grid_search = GridSearchCV(estimator=rfc_param, param_grid=param_grid, cv=5)

    # Fit the grid search object to the data
    grid_search.fit(x_train, y_train)


    # Create a random forest classifier with the best hyperparameters
    rfc = RandomForestClassifier(n_estimators=grid_search.best_params_['n_estimators'], 
                                       min_samples_leaf=grid_search.best_params_['min_samples_leaf'], 
                                       random_state=42)

    # Fit the model on the training data
    rfc.fit(x_train, y_train)

    # Evaluate the model on the test data
    test_score = rfc.score(x_val, y_val)


    # Predict on validation set and calculate accuracy
    y_pred_rf = rfc.predict(x_val)
    accuracy = accuracy_score(y_val, y_pred_rf)

    return y_pred_rf, rfc, accuracy

y_pred_rf, rfc_model, accuracy = random_forest_classification(x_train, y_train, x_val, y_val)
print(y_pred_rf, rfc_model, accuracy)

[210. 208. 210. 208. 208. 208. 210. 210.] RandomForestClassifier(n_estimators=50, random_state=42) 0.75


# Logistic Regression

In [211]:
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import cross_val_score

# Load your dataset and split it into features (X) and target (y)

# Create a logistic regression model
#logreg = LogisticRegression(solver='liblinear', max_iter=1000)

# Use cross-validation to evaluate the model
#scores = cross_val_score(model, x_train, y_train, cv=5)

# Print the mean accuracy score and the standard deviation of the scores
#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


In [212]:
from sklearn.linear_model import LogisticRegression

def find_best_logreg_hyperparameters(x_train, y_train):
    # Define the hyperparameter grid
    param_grid = {'C': [0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

    # Create a logistic regression object
    logreg = LogisticRegression(solver='liblinear', max_iter=1000)

    # Use grid search to find the best combination of hyperparameters
    grid_search = GridSearchCV(logreg, param_grid, cv=5)
    grid_search.fit(x_train, y_train)

    # Return the best hyperparameters and the corresponding accuracy score
    return grid_search.best_params_, grid_search.best_score_

def train_and_evaluate_logreg(x_train, y_train, x_val, y_val, best_params):
    # Create a logistic regression object with the best hyperparameters
    logreg = LogisticRegression(solver='liblinear', max_iter=1000, **best_params)

    # Fit the model to the training data
    logreg.fit(x_train, y_train)

    # Evaluate the model on the validation data
    y_pred_lr = logreg.predict(x_val)
    accuracy = accuracy_score(y_val, y_pred_lr)

    # Return the accuracy score
    return y_pred_lr, accuracy


# Find the best hyperparameters using grid search
best_params, best_score = find_best_logreg_hyperparameters(x_train, y_train)

# Train and evaluate the logistic regression model using the best hyperparameters
y_pred_lr, accuracy = train_and_evaluate_logreg(x_train, y_train, x_val, y_val, best_params)

print(y_pred_lr, accuracy)

[208. 208. 210. 208. 210. 208. 210. 210.] 0.75


# Support Vector Machines

In [213]:
from sklearn.svm import SVC

def svm_classifier(x_train, y_train, x_val, y_val):
    # define the parameter grid
    param_grid = {'C': [0.1, 1, 10]}

    # perform grid search with cross-validation
    svm = SVC()
    grid_search = GridSearchCV(svm, param_grid, cv=5)
    grid_search.fit(x_train, y_train)

    # create an SVM object with the best value of C
    clf_svm = SVC(kernel='linear', C=grid_search.best_params_['C'])
    clf_svm.fit(x_train, y_train)

    # predict on validation data
    y_pred_svm = clf_svm.predict(x_val)

    # calculate accuracy
    accuracy = metrics.accuracy_score(y_val, y_pred_svm)
    return y_pred_svm, clf_svm, accuracy


y_pred_svm, clf_svm, accuracy = svm_classifier(x_train, y_train, x_val, y_val)
print(y_pred_svm, clf_svm, accuracy)

[208. 208. 210. 208. 208. 208. 210. 210.] SVC(C=1, kernel='linear') 0.875


# Naive Bayes

In [214]:
from sklearn.naive_bayes import GaussianNB

def train_and_predict_with_nb(x_train, y_train, x_val, y_val):
    # Build a Gaussian Naive Bayes classifier
    model_nb = GaussianNB()

    # Train the model on the training data
    model_nb.fit(x_train, y_train)

    # Make predictions on the validation set
    y_pred_nb = model_nb.predict(x_val)

    # Evaluate the accuracy of the classifier
    accuracy = metrics.accuracy_score(y_val, y_pred_nb)

    return y_pred_nb, model_nb, accuracy

y_pred_nb, model_nb, accuracy = train_and_predict_with_nb(x_train, y_train, x_val, y_val)
print(y_pred_nb, model_nb, accuracy)

[210. 208. 208. 208. 208. 208. 210. 210.] GaussianNB() 0.625


In [215]:
from sklearn.metrics import f1_score

def compute_f1_scores(y_val, y_pred_dt, y_pred_lr, y_pred_svm, y_pred_rf, y_pred_knn, y_pred_nb):
    # Compute the F1 scores for each model
    dt_f1 = f1_score(y_val, y_pred_dt, average='macro')
    logreg_f1 = f1_score(y_val, y_pred_lr , average='macro')
    svm_f1 = f1_score(y_val, y_pred_svm , average='macro')
    rfc_f1 = f1_score(y_val, y_pred_rf, average='macro')
    knn_f1 = f1_score(y_val, y_pred_knn, average='macro')
    nb_f1 = f1_score(y_val, y_pred_nb, average='macro')
    # Print the F1 scores
    print("Decision Tree F1 score: {:.3f}".format(dt_f1))
    print("Logistic Regression F1 score: {:.3f}".format(logreg_f1))
    print("Support Vector Machine F1 score: {:.3f}".format(svm_f1))
    print("Random Forest F1 score: {:.3f}".format(rfc_f1))
    print("KNN F1 score: {:.3f}".format(knn_f1))
    print("Naive Bayes F1 score: {:.3f}".format(nb_f1))

compute_f1_scores(y_val, y_pred_dt, y_pred_lr, y_pred_svm, y_pred_rf, y_pred_knn, y_pred_nb)

Decision Tree F1 score: 0.750
Logistic Regression F1 score: 0.750
Support Vector Machine F1 score: 0.873
Random Forest F1 score: 0.750
KNN F1 score: 0.619
Naive Bayes F1 score: 0.619
